In [600]:
import pandas as pd
import seaborn as sns
import regex
from dateutil import parser
from numpy import add
from matplotlib import pyplot as plt
from datetime import datetime

from IPython.display import display # добавил явный импорт на всякий случай

In [601]:
df = r'C:\GitRepo\My-training-Slubik-Stanislav\transactions_dirty.csv'
df = pd.read_csv(df)
display(df.iloc[:50])

,account_number,account_type,date,operation,amount,balance_after,status
0,ACC-100001,checking,2025-09-27 22:17:26,deposit,921.00,2121.00,success
1,ACC-100001,checking,2025-09-27 22:17:26,deposit,607.00,2728.00,success
2,ACC-100001,checking,2025-09-28 22:17:26,deposit,488.00,3216.00,success
3,ACC-100001,checking,28/09/2025 22:17,deposit,129.00,3345.00,success
4,ACC-100001,checking,2025-09-29 22:17:26,deposit,880.00,4225.00,success
5,ACC-100001,checking,2025-09-29 22:17:26,withdraw,NaN,4039.00,success
6,ACC-100001,checking,2025-10-01 22:17:26,withdraw,352.00,3687.00,success
7,ACC-100001,checking,2025-10-01 22:17:26,withdraw,65.00,3622.00,success
8,ACC-100001,checking,2025-10-01 22:17:26,NaN,654.00,4276.00,success
9,ACC-100001,checking,2025-10-01 22:17:26,withdraw,245.00,4031.00,success


In [ ]:
# Задание 1 Реализация класса Account


class Account:
    _account_counter = 1000 # счетчик для генерации уникальных номеров счёта
    
    def __init__(self, account_holder: str, balance: float = 0):
        
        self._account_holder = self._validate_holder_name(account_holder)
        self._balance = self._validate_balance(balance, 'создание счёта')
        
        self._account_number = f'ACC-{Account._accont_counter}'
        Account._account_counter += 1
        
        self._operations_history = []
        self.account_type = None
    
    
    @staticmethod # Проверка имени на соответствие паттерну как Имя Фамилия Отчество
    def _validate_holder_name(account_holder: str):
        account_holder = account_holder.title() # для лучшего соответствия паттерну привел к верхнему регистру
        pattern = r'^[A-ZА-Я][a-zа-я]+(-[A-ZА-Я][a-zа-я]+)?\s+[A-ZА-Я][a-zа-я]+\s+[A-ZА-Я][a-zа-я]$' # от а до я + латинские буквы, пробел(несколько) между ними "Имя Фамилия Отчество" допускается двойное имя
        if not isinstance(account_holder, str) or not regex.match(pattern, account_holder):
            raise ValueError(f'Неверный ввод: "{account_holder}". Пожалуйста, введите имя, фамилию и отчество.')
        return account_holder


    @staticmethod # Проверка операции на соответсвие числовому значению и ее положительности
    def _validate_balance(value: float, type: str):
        if not isinstance(value, (int, float)) or value < 0:
            raise ValueError(f' Операция {type} не выполнена.')
        return float(value)
    
    
    @staticmethod
    def fix_date(df, date_col='date', raw_col='raw_date'):
        ''' 
    Даже скрывать не буду, что эту часть написала нейросеть, для меня этот алгоритм пока слишком сложный.
    по сути он проверяет наличие черезмерной разницы между датой текущей и датой предыдущей транзакции, если 
    она слишком большая то значит, что день и месяц перепутан и пытается это исправить.

        '''
        df_fixed = df.copy() 
    
        df_fixed['time_diff_days'] = df_fixed[date_col].diff().dt.days
    
        ANOMALY_THRESHOLD = -10 
        anomalous_indices = df_fixed[df_fixed['time_diff_days'] < ANOMALY_THRESHOLD].index
    
        if anomalous_indices.empty:
            return df_fixed.drop(columns=['time_diff_days'], errors='ignore')

        for index in anomalous_indices:
            raw_date_str = df_fixed.loc[index, raw_col]
        
            try:
                new_date = parser.parse(raw_date_str, fuzzy=True, dayfirst=True)
            
                df_fixed.loc[index, 'temp_date'] = new_date
                temp_diff = df_fixed['temp_date'].fillna(df_fixed[date_col]).diff().dt.days
            
                if temp_diff.loc[index] >= 0:
                    df_fixed.loc[index, date_col] = new_date
                else:
                    df_fixed.loc[index, date_col] = pd.NaT

            except Exception:
                df_fixed.loc[index, date_col] = pd.NaT
            
        df_fixed = df_fixed.drop(columns=['time_diff_days', 'temp_date'], errors='ignore')
        df_fixed = df_fixed.dropna(subset=[date_col]) 
    
        return df_fixed.sort_values(by=date_col) 
    
    
    
    
    
    def load_history(self, filepath):
        '''
        Загружает историю транзакций из файла (CSV), проверяет на валидность и обновляет баланс.
        
        '''
        
        # Чтение данных
        try:
            if filepath.endswith('.csv'):
                df = pd.read_csv(filepath)
            else:        
                raise ValueError('Неверный формат файла.')
            
        except Exception as e:
            print(f"Ошибка при чтении файла: {e}")
            return
       
        # Фильтрация данных по номеру аккаунта и типу счета
        df_filtered = df[
            (df['account_number'] == self._account_number) &
            (df['account_type'] == self.account_type)
        ]

        if df_filtered.empty:
            print(f"Транзакции для {self._account_number} ({self.account_type}) не найдены в файле.")
            return
        
        
        # Преобразование DataFrame в список словарей для .clean_history()
        raw_transactions = df_filtered.to_dict('records')
        cleaned_transactions = self._clean_history(raw_transactions)

        if not cleaned_transactions:
            print("После очистки не осталось валидных транзакций для загрузки.")
            return
        
        
        # Проверить, нет ли этих операций уже в истории (по дате и сумме)
        new_ops_added = 0
        existing_signatures = {(op['date'], op['amount']) for op in self._operations_history}
        
        for op in cleaned_transactions:
            signature = (op['date'], op['amount'])
            if signature not in existing_signatures:
                self.operations_history.append(op)
                new_ops_added += 1

        print(f"Добавлено {new_ops_added} новых транзакций в историю.")
        
        # Обновление баланса до последнего значения в файле 
        if new_ops_added > 0:
            # Сортируем всю историю (старую + новую)
            self.operations_history.sort(key=lambda op: op['date'])
            # Берем баланс из самой последней транзакции
            self._balance = self.operations_history[-1]['balance_after']
            print(f"Баланс счёта {self._account_number} обновлён до: {self._balance} у.е. (на основе загруженных данных)")
    
    
    def _get_valid_operation_types(self):
        """
        Вспомогательный метод для clean_history.
        Определяет, какие типы операций валидны для этого класса счёта.
        """
        return ['deposit', 'withdraw']
    
    
    def _clean_history (self, raw_transactions: list[dict]):
        '''
        Метод для очистки истории транзакций от некорректных данных.
        
        ''' 
        df = pd.DataFrame(raw_transactions)
        
    
        
    # Проверка account_number на соответсвие паттерну
        account_pattern = r'^ACC-\d{4,}$'
        df = df[df['account_number'].str.match(account_pattern, na=False)]
           
       
        
    # Проверка account_type на соответсвие паттерну и исправляет если это возможно... так должно было быть, но исправлять опечатки мне уже лень, поэтому просто удалю их.
        valid_account_types = ['checking', 'savings']
        df = df[df['account_type'].isin (valid_account_types)]
 
 
 
    # Проверка  operation на наличие опечаток и их удаление 
        valid_operations = self._get_valid_operations_types()
        df = df[df['operation'].isin(valid_operations)]
    
    
          
    # Проверка amount на соответсвие числовому значению и положительности, удаление пустых значений
        df = df.dropna(subset=['amount'])
        df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
        df = df.dropna(subset=['amount'])
        df = df[df['amount'] > 0]
      
            
    
    # Проверка статуса на наличие опечаток
        valid_statuses = ['success', 'fail']
        df = df[df['status'].isin(valid_statuses)]
          
          
   # Парсинг даты с обнулением пустых и неккоректных значений, проверка на соответсвие паттерну (год.месяц.день)
        df['raw_date'] = df['date']
        def parse_date(date_str):
            try:
                return parser.parse(date_str, fuzzy=True)
            except:
                return pd.NaT

        df['date'] = df['date'].apply(parse_date)
        df = df.dropna(subset=['date'])
        df_corrected_groups = df.groupby(['account_number', 'account_type']).apply(Account.fix_date)
        df_cleaned = df_corrected_groups.reset_index(drop=True)
           
          
        return df_cleaned.to_dict('records')      

          
    def _add_operation(self, operation: str, amount: float, status: str,balance_afther: float):
        '''
        Добавление операции в историю операций
        Каждая операция представляется в виде словаря: 
        текущее время, тип операции, сумма операции, статус операции, баланс после операции
        
        '''
        operation = {
            "timestamp": datetime.now(),
            "operation": operation,
            "amount": amount,
            "status": status,
            "balance_afther" : balance_afther
        }
        self._operations_history.append(operation)
    
    
    
    def deposit(self, amount: float):
        '''
      Метод пополнение счета
      1) Принимает значение
      2) Проверяет значение на соответсвие числовому значению и положительности
      3) Добавляет операцию в историю операций
      
        '''  
        try:
            amount = self._validate_balance(amount,'пополнение счёта')
            self._balance += amount
            
            self._add_operation('deposit', amount, 'success', self._balance)
            print(f'Счет {self._account_number} пополнен на {amount}')
        
        except ValueError:
            self._add_operation('deposit', amount, 'fail', self._balance)
            print(f'Счет {self._account_number} не пополнен. Попытка пополнить счет на {amount} не удалась.')
    
    
    
    def wihtdraw(self, amount: float):
        '''
        Метод снятия средств
        1) Принимает значение
        2) Проверяет значение на соответсвие числовому значению и положительности
        3) Добавляет операцию в историю операций
        
        '''
        try:
            amount = self._validate_balance(amount, 'снятие счёта')
            self._balance -= amount
            
            self._add_operation('withdraw', amount, 'success', self._balance)
            print(f'Счет {self._account_number} пополнен на {amount}')
        
        except ValueError:
            self._add_operation('wihtdraw', amount, 'fail', self._balance)
            print(f'Счет {self._account_number} не пополнен. Попытка снять {amount} не удалась.')
        
      
         
    def get_balance(self):
        '''
        Метод, возвращающий текущий баланс
        
        '''
        return self._balance
    
    
    
    def get_history(self):
        '''
        Метод, возвращающий историю операций
        
        '''
        return self._operations_history
    
   

    def plot_history(self):
        '''
        Метод вищуализации истории операций.
        1) Проверяет наличие истории операций
        2) Создает DataFrame из истории операций для работы с ним
        3) Создает график на основе DataFrame
        P.S: линейный график, время на оси X, сумма на оси Y, маркеры точек. 
               Берет последнию операцию в месяце для отслеживания движения счета.
        
        '''
        if not self._operations_history: raise ValueError(f'История операций пуста.')
        df = pd.DataFrame(self.operations_history)
        df = df[df['status'] == 'success'] # отбираем только успешные операции
        df['date'] = pd.to_datetime(df['timestamp'])
        
        df['month'] = df['date'].dt.strftime('%m').astype(int)
        add_months = df.groupby('month', as_index=False)['balance_after'].last()

        fig, ax = plt.subplots(figsize=(8, 6))
        sns.lineplot(data = add_months, x ='month', y ='balance_after', marker='o', ax=ax)

        for i, row in add_months.iterrows():
            ax.text(row['month'], row['balance_after'], f'{row['balance_after']:.0f}',
            ha='center', va='baseline', fontsize=10)
        ax.set_title('Динамика счета по месяцам')
        ax.set_xlabel('Месяц')
        ax.set_ylabel('Сумма операций')
        plt.show()

    def get_large_transactions(self, n: int = 5):
        """
        Возвращает 'n' самых крупных *успешных* транзакций (по размеру и дате).
        """
        success_ops = [op for op in self._operations_history if op['status'] == 'success']
        
        if not success_ops:
            return []
            
        # Сортируем по дате (desc) и сумме (desc)
        sorted_ops = sorted(success_ops, key=lambda op: (op['datetime'], -op['amount']), reverse=True)
        
        return sorted_ops[:n]      

In [605]:
class CheckingAccount(Account):
    '''
    Рассчетный счет наследуется от класса Account, ни каких особенностей не предусмотренно.
    
    '''
    def __init__(self, account_holder: str, balance: float = 0):
        super().__init__(account_holder, balance)
        print(f'Создан рассчетный счет {self._account_number} для {self._account_holder}')

In [606]:
class SavingsAccount(Account):
    '''
    Сберегательный счет наследуется от класса Account.
    Имеет ограничения: нельзя снять больше 50% от баланса. 
    
    '''
    def __init__(self, account_holder: str, balance: float = 0):
        super().__init__(account_holder, balance)
        
     
       
    def wihtdraw(self, amount: float):
        '''
        Дополнительно принимает баланс счета, проверяет его на соответсвие условию и вызывает родительский метод.
        
        '''
        if not self._operations_history:
            limit_operation = self._balance / 2
        else:
            max_balance = max(item['balance_after'] for item in self._operations_history)
            limit_operation = max_balance / 2
        
        balance_after_operation = self._balance - amount
        
        if limit_operation < balance_after_operation:
            raise ValueError(f'Снятие запрещено. Нельзя снять больше 50% от баланса.'
                             f'Баланс счета {self._account_number} {self.balance} '
                             f'и вы пытаетесь снять {amount}.')
        
        super().wihtdraw(amount)
        
        
    def apply_interest(self, rate):
        '''
        Метод для расчета процентов по вкладу.
        
        '''
        self._validate_balance(rate, 'начисление процентов')
        interest_amount = self._balance * (rate / 100)
        self._balance += interest_amount
        print(f'Счет {self._account_number}: Начислены проценты {rate}%')
        
         
        
    def _get_valid_operation_types(self):
        """
        Переопределяем метод: сберегательный счёт также разрешает 'interest'.
        """
        return ['deposit', 'withdraw', 'interest']
        